In [1]:
def insta_searching(word):
    url='https://www.instagram.com/explore/tags/'+word
    return url

In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

In [11]:
def insta_searching(word):
    url='https://www.instagram.com/explore/tags/'+word
    return url

In [12]:
def select_first(driver):
    first=driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)


In [116]:
import re
from bs4 import BeautifulSoup
import unicodedata   # 추가, 2020.08.20

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 데이터 기록, 다음 게시물로 클릭
    try: 
        # account 데이터 기록
        #contents = soup.select('span._7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll')[0].text
        #content = unicodedata.normalize('NFC', contents)
        content = soup.find('span',{'class':"_7UhW9.xLCgt.MMzan.KV-D4.se6yk.T0kll"})
        #content = contents.split('#',1)[0]
        #account_data = driver.find_element_by_css_selector('a.sqdOP.yWX7d._8A5w5.ZIAjV')
        #account_text = account_data.text
        
        # 날짜 기록 (주단위)
        date = soup.select('time._1o9PC')[0]['datetime'][:10]
        
        # 날짜 데이터가 시간, 일, 분 단위이면 0주로 변환
        if date.find('시간') != -1 or date.find('일') != -1 or date.find('분') != -1:
            date_text = '0주'
        else:
            date_text = date
        # 해쉬태그 데이터 기록
        data = driver.find_element_by_css_selector('.C7I1f.X7jCj')
        tag_raw = data.text
        tag = re.findall('#[A-Za-z0-9가-힣]+', tag_raw)
        tag = ''.join(tag).replace("#"," ") # "#" 제거
        tag_data = tag.split()

    except:
        tag_data = "error"
        date_text = "error"
        like = 0
    data = [date, content, tag_data]#[content, date, like, tags]
    return data

In [117]:
def move_next(driver):
    right=driver.find_element_by_css_selector('div.l8mY4.feth3')    
    right.click()
    time.sleep(3)

In [118]:
driver=webdriver.Chrome('C:/Users/kaitnam/Downloads/chromedriver_win32/chromedriver.exe')

# 인스타그램 접속하기
driver.get('http://www.instargram.com')
time.sleep(3)

email = 'gfydkate'   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'dldlawkd3' ### 비번 정보 수정 필요
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

# 검색 결과 페이지 열기
word='쏘카'
url=insta_searching(word)
driver.get(url)

time.sleep(3)

# 첫 번째 게시물 클릭
select_first(driver)

# 본격적으로 데이터 수집 시작
results = []

# 수집할 게시물의 수
target = 2

for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

print(results[:2])

[['2022-01-12', None, ['selfie', '셀카', '일상', '운전', '쏘카', '타다', 'trafficjam', 'daily', '드라이브', '여행', 'night', '셀스타그램', '데일리', '셀스타그램', '극한초보', '셀피', '소통', 'ride', 'seoul', '좋아요', '여행스타그램', '꿀잼', '타다패스포트']], ['2021-04-27', None, ['콩코드', '코란도', '기아', '기아자동차', '기아산업', '김경래', '올드카', '클래식카', '출장', '복원', '리스토어', '광고', '협찬', '폐차', '중고차', '술', '대전', '유성', '학하동', '좋아요', '맞팔', '인친', '소통', '팔로우', '다이렉트', '쏘카', '사진', '촬영', '레트로']]]


In [112]:
import pandas as pd

results_df=pd.DataFrame(results)
results_df.columns=['date','content','tag']
results_df

,date,content,tag
0,2022-01-12,타다 패스포트로 쏘카 이용하기🌟-#selfie #셀카 #일상 #운전 #쏘카 #타다 ...,"[selfie, 셀카, 일상, 운전, 쏘카, 타다, trafficjam, daily..."
1,2021-04-27,왕복 500km 출장중에... 발견한콩코드 코란도... .이번주는 내일도 출장 다음...,"[콩코드, 코란도, 기아, 기아자동차, 기아산업, 김경래, 올드카, 클래식카, 출장..."


In [ ]:
import re
from bs4 import BeautifulSoup
import unicodedata   # 추가, 2020.08.20

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content)   # 신규 추가. 2020.08.20
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    #try: 
        #place = soup.select('div.M30cS')[0].text
        #place = unicodedata.normalize('NFC', place)      # 신규 추가. 2020.08.20
    #except:
        #place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, tags]
    return data